<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/Query_Decomposition/claude_prompt_decompose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-aiplatform
!pip install -q -U google-generativeai
!pip install -q groq
!pip install -U accelerate bitsandbytes datasets evaluate
!pip install -U peft transformers trl
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
import time
from tqdm import tqdm

# **Claude**

In [ ]:
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import anthropic
from google.colab import userdata
from sklearn.metrics import precision_recall_fscore_support

# Anthropics API 설정
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)


In [ ]:
system_message = """
You are an experienced Query decomposer.
You need to break down multiple composite queries and change them to a single question.
The reason for dividing a query is to divide it into a single question so that when searching for a query, there are no questions that are marginalized by complex questions.
For example, if you have a query, "What is the CEO name of Apple and Google and Samsung?" you have to divide it into "What is the CEO name of Apple?" "What is the CEO name of Google?" and "What is the CEO name of Samsung?"


I'll provide the query
Write the querys in the following format: 1. query1\n2. query2\n3. query3 ...
If you don't think the query needs to be divided, you don't have to.

Example:
Query: Who was the Super Bowl MVP in 1979 and 1980?
You:
1. Who was the Super Bowl MVP in 1979?
2. Who was the Super Bowl MVP in 1980?

Example:
Query: Are Chris Marker and Yvonne Rainer American?
You:
1. Are Chris Marker American?
2. Are Yvonne Rainer American?

Example:
Query: Does the TechCrunch article on Twitch's subscription revenue split policy indicate a different monetization strategy compared to the TechCrunch article on Beeper's plans for Beeper Mini subscriptions?
You:
1. Does TechCrunch article on Twitch's subscription revenue split policy indicate a monetization strategy?
2. Does TechCrunch article on Beeper's plans for Beeper Mini subscriptions indicate a monetization strategy?

Example:
Query: Which company, as reported by both TechCrunch and The Verge, has spent billions to maintain its default search engine status on various platforms and is also accused of harming news publishers’ revenue through its business practices?
You:
1. Which company, as reported by TechCrunch and The Verge, has spent billions to maintain its default search engine status on various platforms?
2. Which company, as reported by TechCrunch and The Verge, is also accused of harming news publishers’revenue through its business practices?


Do not add anything else when you present the results.
"""

In [ ]:
import anthropic #삭제할 섹션
from google.colab import userdata

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)
que = "Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by both The Verge and TechCrunch, and is accused by prosecutors of committing fraud for personal gain?"
message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=system_message,
    messages=[
        {"role": "user", "content": que}
    ]
)

print(message.content[0].text)

1. Who is the individual associated with the cryptocurrency industry facing a criminal trial on fraud and conspiracy charges, as reported by The Verge and TechCrunch?
2. Who is accused by prosecutors of committing fraud for personal gain in the cryptocurrency industry?


In [ ]:
#결과 쭉 출력
def get_model_response(query):

    message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=system_message,
    messages=[
        {"role": "user", "content": query}
    ]
)
    return message.content[0].text

import json
with open('/content/output_valid.json', 'r') as file:
    data = json.load(file)
for item in data:
        query = item["input"]
        gold_ner = item["output"]
        predicted_ner = get_model_response(query)
        print("Query:", query)
        print("Predicted NER:", predicted_ner)


Query: Query: Were Scott Derrickson and Ed Wood of the same nationality?
Entity: nationality, Scott Derrickson, Ed Wood
Scott Derrickson|nationality|nationality
Ed Wood|nationality|nationality
Predicted NER: 1. What was Scott Derrickson's nationality?
2. What was Ed Wood's nationality?
Query: Query: What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Entity: Corliss Archer, woman, What, film Kiss and Tell, government position
What|was|government position
government position|held by|woman
woman|portrayed|Corliss Archer
Corliss Archer|in|film Kiss and Tell
Predicted NER: 1. Who portrayed Corliss Archer in the film Kiss and Tell?
2. What government position was held by the actress who played Corliss Archer in Kiss and Tell?
Query: Query: What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and alien species?
Entity: stories, enslaved worlds, alien species, fir

In [ ]:
import json
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
def get_model_response(query):

    message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=system_message,
    messages=[
        {"role": "user", "content": query}
    ]
)
    return message.content[0].text

def extract_entities(ner_output):
    # Parse the output to extract entities and their tags
    entities = []
    if ',' in ner_output:
        entities_str = ner_output.split('\n')[-1].strip()
        entities = [tuple(e.strip().split(" ")) for e in entities_str.split(", ")]
    return entities

def get_entities_and_keypoints(query):
    response = get_model_response(query)
    return response

def evaluate_model(test_data):
    y_true = []
    y_pred = []
    incorrect_predictions = []

    for item in test_data:
        query = item["input"]
        gold_ner = item["output"]

        predicted_ner = get_entities_and_keypoints(query)

        gold_entities = extract_entities(gold_ner)
        predicted_entities = extract_entities(predicted_ner)

        y_true.append(set(gold_entities))
        y_pred.append(set(predicted_entities))

        # Check if prediction is incorrect
        if set(gold_entities) != set(predicted_entities):
            incorrect_predictions.append({
                "query": query,
                "gold_entities": gold_entities,
                "predicted_entities": predicted_entities
            })

    mlb = MultiLabelBinarizer()
    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    f1 = f1_score(y_true_bin, y_pred_bin, average='weighted')

    return f1, incorrect_predictions
# Assuming `data` is loaded from the JSON file
with open('/content/output_valid.json', 'r') as file:
    data = json.load(file)

# Evaluate the model and get incorrect predictions
f1, incorrect_predictions = evaluate_model(data)
print(f"F1 Score: {f1}")

# Output incorrect predictions
for incorrect in incorrect_predictions:
    print("Query:", incorrect["query"])
    print("Gold Entities:", incorrect["gold_entities"])
    print("Predicted Entities:", incorrect["predicted_entities"])
    print()

F1 Score: 0.0
Query: Query: Were Scott Derrickson and Ed Wood of the same nationality?
Entity: nationality, Scott Derrickson, Ed Wood
Scott Derrickson|nationality|nationality
Ed Wood|nationality|nationality
Gold Entities: [('Scott', 'Derrickson', '(etc)'), ('Ed', 'Wood', '(etc)'), ('nationality', '(core)')]
Predicted Entities: []

Query: Query: What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Entity: Corliss Archer, woman, What, film Kiss and Tell, government position
What|was|government position
government position|held by|woman
woman|portrayed|Corliss Archer
Corliss Archer|in|film Kiss and Tell
Gold Entities: [('Corliss', 'Archer', '(etc)'), ('woman', '(core)'), ('What', '(etc)'), ('film', 'Kiss', 'and', 'Tell', '(etc)'), ('government', 'position', '(etc)')]
Predicted Entities: []

Query: Query: What science fantasy young adult series, told in first person, has a set of companion books narrating the stories of enslaved worlds and 

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [('1.', 'According', 'to', 'the', '2001', 'census'), ('1.', 'What', 'coastal', 'area', 'does', 'a', 'medieval', 'fortress', 'in', 'Dirleton'), ('1.', 'When', 'was', "Poison's", 'album', '"Shut', 'Up'), ('1.', 'Who', 'is', 'younger'), ('2.', 'In', 'how', 'many', 'countries', 'does', 'this', 'corporation', 'have', 'operations?'), ('3.', 'Where', 'are', 'the', 'branch', 'campuses', 'of', 'the', 'university', 'whose', 'main', 'campus', 'is', 'in', 'Lawrence'), ('3.', 'Which', 'has', 'a', 'higher', 'instrument', 'to', 'person', 'ratio'), ('3.', 'Who', 'is', 'older'), ('Annie', 'Morton', 'or', 'Terry', 'Richardson?'), ('Badly', 'Drawn', 'Boy', 'or', 'Wolf', 'Alice?'), ('East', 'Lothian'), ('Kansas', 'located?'), ('Keith', 'Bostic', 'or', 'Jerry', 'Glanville?'), ('Make', 'Love"', 'released?'), ('Scotland', 'border', 'on', 'the', 'south', 'side?'), ('what', 'was', 'the', 'population', 'o

In [ ]:
import anthropic
from google.colab import userdata
NER_prompt = """
You are a capable entity extractor.
You need to extract all Entities from the given sentence (query, question).
You need to extract all the entities that have meaning in the sentence.
All Entities should be extracted in the form of Entities, Entities, Entities.
When extracting entity, extract as much as possible.
interrogative word must should be treated as an entity Where was it located.
Don't print anything other than what you asked

e.g. )
Question: What measures might the international community take if X (formerly Twitter) fails to comply with the European Union's Code?

->
What, measures, international community, X (formerly Twitter), European Union's Code

e.g. )
Question: Who was the Super Bowl MVP in 1979 and 1980.

->
Who, Super Bowl MVP, 1979 and 1980

e.g. )
Question: 2014 S/S is the debut album of a South Korean boy group that was formed by who?

->
2014 S/S, debut album, South Korean boy group, who
"""
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1000,
    temperature=0.0,
    system=NER_prompt,
    messages=[
        {"role": "user", "content": "query"}
    ]
)

print(message.content[0].text)

I apologize, but you haven't provided an actual query or question for me to extract entities from. Could you please provide a specific sentence or question that you'd like me to analyze for entities? Once you do, I'll be happy to extract the relevant entities for you.
